# Connecting Python to PostgreSQL with `psycopg2`

This notebook is the foundation for interacting with PostgreSQL from Python. We will learn how to use the `psycopg2` library, the most popular PostgreSQL database adapter for Python.

We will cover:
1.  Installing the `psycopg2` library.
2.  Establishing a `connection` to the database.
3.  Creating a `cursor` to execute commands.
4.  Running a simple `SELECT` query and fetching results.
5.  Using the `with` statement for safe and clean resource management.

--- 
## Step 1: Install `psycopg2`

First, we need to install the library. `psycopg2-binary` is the easiest way to get started as it includes all the necessary dependencies.

In [1]:
!pip install psycopg2-binary

--- 
## Step 2: Connecting to the Database

To connect, we import the library and use `psycopg2.connect()`. This function returns a **connection object**.

The **connection** represents your entire session with the database. You use it to manage transactions (`commit`, `rollback`) and create cursors.

In [2]:
import psycopg2
import os # Used for securely getting credentials in a real app

# --- Best Practice: In a real application, NEVER hardcode credentials. ---
# Use environment variables, a secrets manager, or a config file.
# For this notebook, we'll define them here for clarity.
DB_HOST = "localhost"
DB_NAME = "people"
DB_USER = "fahad"
DB_PASS = "secret"

conn = None
try:
    conn = psycopg2.connect(
        host=DB_HOST,
        dbname=DB_NAME,
        user=DB_USER,
        password=DB_PASS
    )
    print("Connection successful!")
except psycopg2.OperationalError as e:
    print(f"Could not connect to the database: {e}")

Connection successful!


--- 
## Step 3: Creating a Cursor and Executing a Query

Once connected, you need a **cursor object** to execute commands. Think of the cursor as your active tool for interacting with the database tables. You can have multiple cursors per connection.

We'll create a cursor, execute a simple query to get the PostgreSQL version, and fetch the result.

In [3]:
if conn:
    cur = conn.cursor()
    
    # Execute a command
    cur.execute('SELECT version();')
    
    # Retrieve the result
    db_version = cur.fetchone()
    print(f"Database version: {db_version[0]}")
    
    # --- ALWAYS close your cursor and connection when done --- 
    cur.close()
    conn.close()
    print("Connection closed.")

Database version: PostgreSQL 17.6 on x86_64-windows, compiled by msvc-19.44.35213, 64-bit
Connection closed.


--- 
## Step 4: The 'Pythonic' Way with `with` Statements

Manually closing cursors and connections is risky. If an error occurs before the `.close()` calls, the connection might be left open, consuming resources.

Python's `with` statement automatically handles closing resources, even if errors happen. This is the recommended, modern way to work with `psycopg2`.

In [4]:
try:
    # The 'with' statement handles conn.close() for us
    with psycopg2.connect(host=DB_HOST, dbname=DB_NAME, user=DB_USER, password=DB_PASS) as conn:
        
        # The 'with' statement handles cur.close() for us
        with conn.cursor() as cur:
            
            cur.execute('SELECT COUNT(*) FROM pg_catalog.pg_tables;')
            table_count = cur.fetchone()[0]
            print(f"Total tables in the database schema: {table_count}")

    print("Connection closed automatically.")
except psycopg2.OperationalError as e:
    print(f"Could not connect: {e}")

Total tables in the database schema: 103
Connection closed automatically.


--- 
## Conclusion

In this notebook, we learned the fundamental pattern for connecting Python to PostgreSQL:

1.  **Import** `psycopg2`.
2.  Use `psycopg2.connect()` to get a **connection**.
3.  Use `connection.cursor()` to get a **cursor**.
4.  Use `cursor.execute()` to run SQL commands.
5.  Use `cursor.fetchone()` or `cursor.fetchall()` to retrieve data.
6.  Wrap everything in `with` statements for safe and automatic resource management.

With this foundation, we are ready to perform more complex database operations.